In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from collections import defaultdict
from array import array
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import math
import numpy as np
import collections
from numpy import linalg as la
from numpy.linalg import norm
import pandas as pd
import string
import re

import nltk
nltk.download('stopwords')

!pip3 install rank-bm25
from rank_bm25 import BM25Okapi

!pip3 install gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 42.2 MB/s eta 0:00:00


In [3]:
# Introduce your path to fashion_products_dataset.json

# Pau Path
#docs_path = '/content/drive/MyDrive/4 curs/Recuperació de la info/Final Project/fashion_products_dataset.json'

# Marc Path
docs_path = '/content/drive/MyDrive/Final Project/fashion_products_dataset.json'

# Clara Path
# docs_path = '/content/drive/MyDrive/4t/Recuperació de la Informació i Anàlisi de la Web/Final Project/fashion_products_dataset.json'

# Teacher Path - UPDATE WITH YOUR PATH
#docs_path =

### **From the first two parts we have only left the code necessary to continue with part 3.**



#Part 1: Text Processing and Exploratory Data Analysis

##PART 1.1: Data preparation

In [4]:
# Load the JSON data into a DataFrame
df = pd.read_json(docs_path)

display(df.head())

,_id,actual_price,average_rating,brand,category,crawled_at,description,discount,images,out_of_stock,pid,product_details,seller,selling_price,sub_category,title,url
0,fa8e22d6-c0b6-5229-bb9e-ad52eda39a0a,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:51,Yorker trackpants made from 100% rich combed c...,69% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EA7H5FYZH,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,921,Bottomwear,Solid Women Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
1,893e6980-f2a0-531f-b056-34dd63fe912c,"1,499",3.9,York,Clothing and Accessories,2021-02-10 20:11:52,Yorker trackpants made from 100% rich combed c...,66% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EJZV2UVRZ,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,499,Bottomwear,Solid Men Blue Track Pants,https://www.flipkart.com/yorker-solid-men-blue...
2,eb4c8eab-8206-59d0-bcd1-a724d96bf74f,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:52,Yorker trackpants made from 100% rich combed c...,68% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EHFCY5Z4Y,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,931,Bottomwear,Solid Men Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
3,3f3f97bb-5faf-57df-a9ff-1af24e2b1045,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:53,Yorker trackpants made from 100% rich combed c...,69% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9ESZZ7YWEF,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Shyam Enterprises,911,Bottomwear,Solid Women Multicolor Track Pants,https://www.flipkart.com/yorker-solid-men-mult...
4,750caa3d-6264-53ca-8ce1-94118a1d8951,"2,999",3.9,York,Clothing and Accessories,2021-02-10 20:11:53,Yorker trackpants made from 100% rich combed c...,68% off,[https://rukminim1.flixcart.com/image/128/128/...,False,TKPFCZ9EVXKBSUD7,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Shyam Enterprises,943,Bottomwear,"Solid Women Brown, Grey Track Pants",https://www.flipkart.com/yorker-solid-men-brow...


**1.** In this section, we will prepare the text data for analysis. This involves several key steps:

  1.  **Removing stop words**: Eliminating common words like "the", "a", and "is" that don't add much meaning.
  2.  **Tokenization**: Breaking down the text into individual words or tokens.
  3.  **Removing punctuation marks**: Cleaning the text by getting rid of symbols like commas, periods, and exclamation points.
  4.  **Stemming**: Reducing words to their root form (e.g., "running" becomes "run").
  5.  **Other preprocessing**: We will also consider other steps like converting text to lowercase and handling potential special characters.


In [5]:
def token_cleaning_text(text):
    stemmer = PorterStemmer()
    stop_words = set(stopwords.words("english"))
    #put everything in lowercase
    text=  text.lower()
    #get rid of punctuation marks
    text = text.translate(str.maketrans('', '', string.punctuation))
    #remove special characters
    text = re.sub(r'[^a-z0-9\s]', '', text)
    #tokenize the sentence
    text=  text.split()
    #filter words to delete stop words and stemming them
    text=[word for word in text if word not in stop_words]
    text=[stemmer.stem(word) for word in text]
    return text

In [6]:
#apply the token_cleaning_text function to each value in the 'title' and 'description' columns and store the cleaned version in a new column
df['cleaned_title'] = df['title'].apply(token_cleaning_text)
df['cleaned_description'] = df['description'].apply(token_cleaning_text)

#combine the cleaned title and cleaned description columns into a single text field
df['cleaned_title_description'] = df['cleaned_title'] + df['cleaned_description']

display(df[['title', 'cleaned_title', 'description', 'cleaned_description', 'cleaned_title_description']].head())

,title,cleaned_title,description,cleaned_description,cleaned_title_description
0,Solid Women Multicolor Track Pants,"[solid, women, multicolor, track, pant]",Yorker trackpants made from 100% rich combed c...,"[yorker, trackpant, made, 100, rich, comb, cot...","[solid, women, multicolor, track, pant, yorker..."
1,Solid Men Blue Track Pants,"[solid, men, blue, track, pant]",Yorker trackpants made from 100% rich combed c...,"[yorker, trackpant, made, 100, rich, comb, cot...","[solid, men, blue, track, pant, yorker, trackp..."
2,Solid Men Multicolor Track Pants,"[solid, men, multicolor, track, pant]",Yorker trackpants made from 100% rich combed c...,"[yorker, trackpant, made, 100, rich, comb, cot...","[solid, men, multicolor, track, pant, yorker, ..."
3,Solid Women Multicolor Track Pants,"[solid, women, multicolor, track, pant]",Yorker trackpants made from 100% rich combed c...,"[yorker, trackpant, made, 100, rich, comb, cot...","[solid, women, multicolor, track, pant, yorker..."
4,"Solid Women Brown, Grey Track Pants","[solid, women, brown, grey, track, pant]",Yorker trackpants made from 100% rich combed c...,"[yorker, trackpant, made, 100, rich, comb, cot...","[solid, women, brown, grey, track, pant, yorke..."


**2.** We will store the important attributes in a array for future returning operations.

In [7]:
important_attributes = [
    'pid',
    'title',
    'description',
    'brand',
    'category',
    'sub_category',
    'product_details',
    'seller',
    'out_of_stock',
    'selling_price',
    'discount',
    'actual_price',
    'average_rating',
    'url'
]

#keep only the important attributes for retrieval
df_important_attributes = df[important_attributes].copy()

display(df_important_attributes.head())

,pid,title,description,brand,category,sub_category,product_details,seller,out_of_stock,selling_price,discount,actual_price,average_rating,url
0,TKPFCZ9EA7H5FYZH,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",Shyam Enterprises,False,921,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...
1,TKPFCZ9EJZV2UVRZ,Solid Men Blue Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",Shyam Enterprises,False,499,66% off,"1,499",3.9,https://www.flipkart.com/yorker-solid-men-blue...
2,TKPFCZ9EHFCY5Z4Y,Solid Men Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",Shyam Enterprises,False,931,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...
3,TKPFCZ9ESZZ7YWEF,Solid Women Multicolor Track Pants,Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",Shyam Enterprises,False,911,69% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-mult...
4,TKPFCZ9EVXKBSUD7,"Solid Women Brown, Grey Track Pants",Yorker trackpants made from 100% rich combed c...,York,Clothing and Accessories,Bottomwear,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",Shyam Enterprises,False,943,68% off,"2,999",3.9,https://www.flipkart.com/yorker-solid-men-brow...


**3.** We will start by separating the different fields that we have in product_details. We will create a new attribute called product_{attribute} for each of them.

In [8]:
def extract_product_detail(product_details_list, key):
    if isinstance(product_details_list, list):
        for detail_dict in product_details_list:
            if isinstance(detail_dict, dict) and key in detail_dict:
                return detail_dict[key]
    # return None if the key is not found
    return None

#possible product attributes
producte_attributes = ["Style Code", "Closure", "Pockets", "Fabric", "Pattern", "Color"]

#create new columns by applying the extraction function
for attribute in producte_attributes:
    df[f'product_{attribute.lower().replace(" ", "_")}'] = df['product_details'].apply(lambda x: extract_product_detail(x, attribute))

#display the new columns along with the original product_details
display(df[['product_details'] + [f'product_{key.lower().replace(" ", "_")}' for key in producte_attributes]].head())

,product_details,product_style_code,product_closure,product_pockets,product_fabric,product_pattern,product_color
0,"[{'Style Code': '1005COMBO2'}, {'Closure': 'El...",1005COMBO2,Elastic,Side Pockets,Cotton Blend,Solid,Multicolor
1,"[{'Style Code': '1005BLUE'}, {'Closure': 'Draw...",1005BLUE,"Drawstring, Elastic",Side Pockets,Cotton Blend,Solid,Blue
2,"[{'Style Code': '1005COMBO4'}, {'Closure': 'El...",1005COMBO4,Elastic,Side Pockets,Cotton Blend,Solid,Multicolor
3,"[{'Style Code': '1005COMBO3'}, {'Closure': 'El...",1005COMBO3,Elastic,Side Pockets,Cotton Blend,Solid,Multicolor
4,"[{'Style Code': '1005COMBO1'}, {'Closure': 'Dr...",1005COMBO1,"Drawstring, Elastic",Side Pockets,Cotton Blend,Solid,"Brown, Grey"


Process the fields (category, sub_category, fabric, pattern, and color) by cleaning and merging them with the main token list for TF-IDF future analysis.

In [9]:
for col in ['category', 'sub_category', 'product_fabric', 'product_pattern', 'product_color']:
    #replace NaN, None, or empty values with empty string before applying the function
    df[col] = df[col].fillna('').astype(str).replace('None', '').replace('nan', '')
    df[f'cleaned_{col}'] = df[col].apply(token_cleaning_text)

df['cleaned_title_description_extra_fields'] = (
    df['cleaned_title_description']
    + df['cleaned_category']
    + df['cleaned_sub_category']
    + df['cleaned_product_fabric']
    + df['cleaned_product_pattern']
    + df['cleaned_product_color']
)

display(df[['cleaned_category', 'cleaned_sub_category', 'cleaned_product_fabric', 'cleaned_product_pattern', 'cleaned_product_color', 'cleaned_title_description_extra_fields']].head())

,cleaned_category,cleaned_sub_category,cleaned_product_fabric,cleaned_product_pattern,cleaned_product_color,cleaned_title_description_extra_fields
0,"[cloth, accessori]",[bottomwear],"[cotton, blend]",[solid],[multicolor],"[solid, women, multicolor, track, pant, yorker..."
1,"[cloth, accessori]",[bottomwear],"[cotton, blend]",[solid],[blue],"[solid, men, blue, track, pant, yorker, trackp..."
2,"[cloth, accessori]",[bottomwear],"[cotton, blend]",[solid],[multicolor],"[solid, men, multicolor, track, pant, yorker, ..."
3,"[cloth, accessori]",[bottomwear],"[cotton, blend]",[solid],[multicolor],"[solid, women, multicolor, track, pant, yorker..."
4,"[cloth, accessori]",[bottomwear],"[cotton, blend]",[solid],"[brown, grey]","[solid, women, brown, grey, track, pant, yorke..."


**4.** The fields out_of_stock, selling_price, discount, actual_price, and average_rating were converted to their appropriate data types to enable accurate filtering and numerical operations.

In [10]:
#convert 'actual_price', 'selling_price and 'average_rating' to numeric, handling commas and null values
df['actual_price'] = df['actual_price'].str.replace(',', '', regex=False)
df['actual_price'] = pd.to_numeric(df['actual_price'], errors='coerce')

df['selling_price'] = df['selling_price'].str.replace(',', '', regex=False)
df['selling_price'] = pd.to_numeric(df['selling_price'], errors='coerce')

df['average_rating'] = pd.to_numeric(df['average_rating'], errors='coerce')

print(df[['actual_price', 'selling_price', 'average_rating']].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28080 entries, 0 to 28079
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   actual_price    27303 non-null  float64
 1   selling_price   28078 non-null  float64
 2   average_rating  25819 non-null  float64
dtypes: float64(3)
memory usage: 658.3 KB
None


In [11]:
#clean and convert the discount
df['discount'] = df['discount'].str.replace(' off', '', regex=False)
df['discount'] = df['discount'].str.replace('%', '', regex=False)
df['discount'] = pd.to_numeric(df['discount'], errors='coerce')
df['discount'] = df['discount'].astype('Int64')

print(df['discount'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 28080 entries, 0 to 28079
Series name: discount
Non-Null Count  Dtype
--------------  -----
27225 non-null  Int64
dtypes: Int64(1)
memory usage: 246.9 KB
None


We can see that some values have null values now we transformed to numerical some data that was stored as strings. For example the discount that now have null values were initially as an empty string. We can see what they looked like before the cleaning.

In [12]:
#find rows where 'discount' is null after conversion
null_discount_rows = df[df['discount'].isnull()]

#display these rows, including the original 'discount' column from df_important_attributes
display(df_important_attributes.loc[null_discount_rows.index, ['title', 'discount', 'actual_price', 'selling_price']].head(20))

,title,discount,actual_price,selling_price
1705,Solid Women Polo Neck Blue T-Shirt,,,"1,099"
1734,Men Striped Ankle Length,,,499
1891,Solid Women Grey Track Pants,,,"1,499"
1922,Full Sleeve Solid Men Sweatshirt,,,"2,399"
1949,Solid Men Grey Track Pants,,,"1,499"
1950,Solid Men Polo Neck Green T-Shirt,,,"1,299"
1953,Full Sleeve Solid Men Sports Jacket,,,"3,699"
1958,Solid Men Polo Neck Green T-Shirt,,,"1,599"
1964,Original Cotton Half Cushion Women Ankle Lengt...,,,399
2020,Printed Women Round Neck Dark Blue T-Shirt,,,"3,999"


For more cleaned data we will assume that there was no discount so we will just set the actual_price to have the same value as the selling_price and we will put the discount to 0.

In [13]:
#fill null discount values with 0
df['discount'] = df['discount'].fillna(0)

#for rows where discount was null (and now 0), set actual_price equal to selling_price
df.loc[df['discount'] == 0, 'actual_price'] = df['selling_price']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28080 entries, 0 to 28079
Data columns (total 32 columns):
 #   Column                                  Non-Null Count  Dtype         
---  ------                                  --------------  -----         
 0   _id                                     28080 non-null  object        
 1   actual_price                            28078 non-null  float64       
 2   average_rating                          25819 non-null  float64       
 3   brand                                   28080 non-null  object        
 4   category                                28080 non-null  object        
 5   crawled_at                              28080 non-null  datetime64[ns]
 6   description                             28080 non-null  object        
 7   discount                                28080 non-null  Int64         
 8   images                                  28080 non-null  object        
 9   out_of_stock                            28080 non-

# Part 2: Indexing and evaluation



## 2.1 Indexing



### 2.1.1 Inverted index

The cleaned data is in the variable "df", which contains the data as we got it and the new transformed features we created by cleaning and treating the data, so we will use this dataframe.

We will use the merged text feature we created in the first part.

In [14]:
def create_index(data):
    index = defaultdict(list) # store unique document IDs
    title_index = {} # dictionary to map page titles to page ids

    for _, row in data.iterrows(): # iterate over DataFrame rows
        doc_id = row['pid'] # 'pid' is the document ID
        title = row['title'] # 'title' is the document title
        title_index[doc_id] = title

        # use the cleaned_title_description_extra_fields for indexing
        terms = row['cleaned_title_description_extra_fields']
        current_page_index = {}

        for position, term in enumerate(terms): # terms contains all the tokens from cleaned_title_description_extra_fields column

            try:
                # if the term is already in the index for the current page (current_page_index)
                # append the position to the corresponding list
                current_page_index[term][1].append(position)
            except:
                # Add the new term as dict key and initialize the array of positions and add the position
                current_page_index[term]=[doc_id, array('I',[position])] #'I' indicates unsigned int (int in Python)

        #merge the current page index with the main index
        for term, posting in current_page_index.items():
          index[term].append(posting)

    return index, title_index

In [15]:
inverted_index, title_index = create_index(df)
print("Inverted Index:")
for term, doc_ids in list(inverted_index.items())[:20]:
    print(f"{term}: {doc_ids}")

print("\nTitle Index:")
for doc_id, title in list(title_index.items())[:20]:
    print(f"{doc_id}: {title}")

Inverted Index:
solid: [['TKPFCZ9EA7H5FYZH', array('I', [0, 31])], ['TKPFCZ9EJZV2UVRZ', array('I', [0, 31])], ['TKPFCZ9EHFCY5Z4Y', array('I', [0, 31])], ['TKPFCZ9ESZZ7YWEF', array('I', [0, 31])], ['TKPFCZ9EVXKBSUD7', array('I', [0, 32])], ['TKPFCZ9EFK9DNWDA', array('I', [0, 31])], ['TKPFDABN3GXYPFHE', array('I', [0, 32])], ['TKPFCZ9ESGZYT8NH', array('I', [0, 31])], ['TKPFCZ9DYU33FFXS', array('I', [0, 31])], ['TKPFDABN4NQFVKZY', array('I', [0, 31])], ['TKPFZFSHHACG3FHC', array('I', [0, 10])], ['TKPFZFSHQPDRGZTM', array('I', [0, 10])], ['TKPFZFSH3F9ZA7C6', array('I', [0, 10])], ['TKPFCZ9EHCNAPKPU', array('I', [0, 31])], ['TKPFDACEXAWUHGR7', array('I', [0, 32])], ['TKPFD3K62JB9PEMR', array('I', [0, 31])], ['TKPFCZ9EZDPZR5AH', array('I', [0, 31])], ['TKPFWBGVGU9FCAYX', array('I', [0, 11])], ['TKPFCZ9EVM2GZ4GF', array('I', [0, 31])], ['TKPFWAG7YFWPMG5Y', array('I', [0, 10])], ['TKPFCZ9E2UC3DR3F', array('I', [0, 32])], ['TKPFCZ9ECDYYDNKA', array('I', [0, 32])], ['SWTFYHP2QQQYEGRJ', array('I'

### 2.1.2 Test query proposals, TF-IDF algorithm implementation and product ranking results:

To choose the queries we first want to visualize the terms based on popularity, frequency and TF-IDF results, in this way we can select the ones we believe are most appropriate. To do this, we must implement the create_index_tfidf function that will do the same as the creation of the previous index, but will also calculate the term frequency, document frequency and the inverse values ​​of the document frequency for each token. We can print these results to see how to make creative queries that could be real and reasonable for a user's query.

In [16]:
def create_index_tfidf(data):

    num_documents = len(data)
    index = defaultdict(list)
    tf = defaultdict(list)
    df = defaultdict(int)
    idf = defaultdict(float)

    for _, row in data.iterrows():
        doc_id = row["pid"]
        terms = row["cleaned_title_description_extra_fields"]
        current_page_index = {}
        for position, term in enumerate(terms):
            try:
                current_page_index[term][1].append(position)
            except:
                current_page_index[term] = [doc_id, array('I', [position])]

        norm = 0
        for term, posting in current_page_index.items():

            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term
            tf[term].append(np.round(len(posting[1]) / norm, 4))
            #increment the doc frequency of current term (number of documents containing the current term)
            df[term] += 1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
    # Compute IDF
    for term in df:
        idf[term] = np.round(np.log(float(num_documents / df[term])), 4)
    return index, tf, df, idf


We call the new index function to store the tf, df and idf values, which will be useful in the following steps.

In [17]:
index, tf, df_sol, idf = create_index_tfidf(df)

To rank the documents/products for each query, we used the same function as in Lab 2, but we added the following condition: conjunctive queries (AND). This means that every returned document/product must contain all the words from the query in order to be considered a match.

Therefore, we first go through all the terms in the query and store in a general list all the documents that contain at least one of the terms. Then, we perform an intersection to keep only the documents that contain all the terms. In this way, we ensure that only documents meeting this condition are added to the doc_vectors list, which contains the relevant documents for a given query.

In [18]:
def rank_documents(terms, docs, index, idf, tf):
    result_docs = []
    doc_scores = []

    #for each term in the query (terms) obtain the set of documents that contain it
    docs_with_query_terms = []
    for term in terms:
        if term in index:
          #the term appears in at least one document because it's present in the index
          #get all documents where the term appears
          term_docs = {doc for doc, _ in index[term]}
          docs_with_query_terms.append(term_docs)

    # if no query terms are found in the index, there are no matching documents
    if not docs_with_query_terms:
        print("No query terms in the index")
        return result_docs, doc_scores

    # intersection, keep only documents that contain all query terms
    docs_with_all_query_terms = set.intersection(*docs_with_query_terms)
    if not docs_with_all_query_terms:
        print("No documents with all query terms")
        return result_docs, doc_scores

    #interested only on the element of the docVector corresponding to the query terms
    # remaining elements 0
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query.

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):

            #tf[term][0] will contain the tf of the term "term" in the doc 26
            if doc in docs_with_all_query_terms:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # check if multiply for idf

    # Calculate the score of each doc, cosine similarity between queyVector and each docVector

    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)

    return result_docs, doc_scores

In [19]:
def search_tf_idf(query, index):

    query = token_cleaning_text(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"
            term_docs=[posting[0] for posting in index[term]]

            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs, doc_scores = rank_documents(query, docs, index, idf, tf)
    return ranked_docs, doc_scores

In [20]:
test_queries = [
    "cotton t-shirt men",
    "women winter jacket",
    "sports shoes running",
    "formal shirt slim fit",
    "casual footwear black"
]

print("Proposed test queries:")
for q in test_queries:
    print("-", q)

Proposed test queries:
- cotton t-shirt men
- women winter jacket
- sports shoes running
- formal shirt slim fit
- casual footwear black


In [21]:
for q in test_queries:
    print(f"\nQuery: {q}")
    ranked_docs, _ = search_tf_idf(q, index)
    print(f"Top results for '{q}': {ranked_docs[:10]}")



Query: cotton t-shirt men
Top results for 'cotton t-shirt men': ['TSHFHTGBXB3ENHWH', 'TSHFHTGBHE7HGXKT', 'TSHFVR8F4QDGJAP4', 'TSHFDVE6JNWBGJ2W', 'TSHFGZKNRCNSQ64A', 'TSHFGZKNBYBZ9FAJ', 'TSHFN3V5FBWMFE2J', 'TSHFN3V5BH32EZQF', 'TSHFN3V59WS2G8U4', 'TSHFF75PHCFAHE7Z']

Query: women winter jacket
Top results for 'women winter jacket': ['JCKFWYTHH3B5GKWS', 'JCKFWYNCWEHNZCTJ', 'JCKFW7ERKBA4BHY3', 'JCKFVVHNUHUHMYSN', 'JCKFVVHNHW89CS4E', 'JCKFVVHNG3ME8HYZ', 'JCKFVVHNFQYBYNGH', 'JCKFVVHNFANGGWNP', 'JCKFVVHN72FTTT9B', 'JCKFU9APAQJBN23B']

Query: sports shoes running
Top results for 'sports shoes running': ['SHOFN3EJZVGC6GH8', 'SHOFN3EJVGC9VXYT', 'SHOFN3EJHXHGSGUC', 'SHOFN3EFNYGBKZT7', 'SHOFUH9JBHVZC5TU', 'SHOFG5TXSM7M57MY', 'SHOF98H4X8PU3EGW', 'SHOFVGHHVVUZGFA6', 'SHOFUWMJ9CMSKHUD', 'SHOFUH52SH3EQDH5']

Query: formal shirt slim fit
Top results for 'formal shirt slim fit': ['SHTFDYVVNDXJQWWV', 'SHTFDYU42BPWHM3Y', 'SHTFDYRQRYGTW4PX', 'SHTFDYPHFAB6UKF8', 'SHTFDYSQZ9AG7ZTV', 'SHTFW53YCYWCJGHD', 'SHT

As you can see, there are more than 10 results for these queries, which makes them reasonable, there are products that can meet these requirements.

# Part 3: Ranking and Filtering

In this part of the project, we experimented with different ranking algorithms that can be applied in a search engine. Our goal was to explore and compare several relevance scoring methods in order to understand how each algorithm influences the final ranking of the retrieved documents.

## Three different ways of rankings


### TF-IDF and cosine similarity


We reused the functions developed in Part 2 to compute TF-IDF representations and apply cosine similarity for document ranking.

In [22]:
def create_index_tfidf(data):

    num_documents = len(data)
    index = defaultdict(list)
    tf = defaultdict(list)
    df = defaultdict(int)
    idf = defaultdict(float)

    for _, row in data.iterrows():
        doc_id = row["pid"]
        terms = row["cleaned_title_description_extra_fields"]
        current_page_index = {}
        for position, term in enumerate(terms):
            try:
                current_page_index[term][1].append(position)
            except:
                current_page_index[term] = [doc_id, array('I', [position])]

        norm = 0
        for term, posting in current_page_index.items():

            norm += len(posting[1]) ** 2
        norm = math.sqrt(norm)
        #calculate the tf(dividing the term frequency by the above computed norm) and df weights
        for term, posting in current_page_index.items():
            # append the tf for current term
            tf[term].append(np.round(len(posting[1]) / norm, 4))
            #increment the doc frequency of current term (number of documents containing the current term)
            df[term] += 1 # increment DF for current term

        #merge the current page index with the main index
        for term_page, posting_page in current_page_index.items():
            index[term_page].append(posting_page)
    # Compute IDF
    for term in df:
        idf[term] = np.round(np.log(float(num_documents / df[term])), 4)
    return index, tf, df, idf


We call the new index function to store the tf, df and idf values, which will be useful in the following steps.

In [23]:
index, tf, df_sol, idf = create_index_tfidf(df)

To rank the documents/products for each query, we used the same function as in Lab 2, but we added the following condition: conjunctive queries (AND). This means that every returned document/product must contain all the words from the query in order to be considered a match.

Therefore, we first go through all the terms in the query and store in a general list all the documents that contain at least one of the terms. Then, we perform an intersection to keep only the documents that contain all the terms. In this way, we ensure that only documents meeting this condition are added to the doc_vectors list, which contains the relevant documents for a given query

In [24]:
def rank_documents(terms, docs, index, idf, tf):
    result_docs = []
    doc_scores = []

    #for each term in the query (terms) obtain the set of documents that contain it
    docs_with_query_terms = []
    for term in terms:
        if term in index:
          #the term appears in at least one document because it's present in the index
          #get all documents where the term appears
          term_docs = {doc for doc, _ in index[term]}
          docs_with_query_terms.append(term_docs)

    # if no query terms are found in the index, there are no matching documents
    if not docs_with_query_terms:
        print("No query terms in the index")
        return result_docs, doc_scores

    # intersection, keep only documents that contain all query terms
    docs_with_all_query_terms = set.intersection(*docs_with_query_terms)
    if not docs_with_all_query_terms:
        print("No documents with all query terms")
        return result_docs, doc_scores

    #interested only on the element of the docVector corresponding to the query terms
    # remaining elements 0
    doc_vectors = defaultdict(lambda: [0] * len(terms)) # I call doc_vectors[k] for a nonexistent key k, the key-value pair (k,[0]*len(terms)) will be automatically added to the dictionary
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)  # get the frequency of each term in the query.

    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):  #termIndex is the index of the term in the query
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):

            #tf[term][0] will contain the tf of the term "term" in the doc 26
            if doc in docs_with_all_query_terms:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]  # check if multiply for idf

    # Calculate the score of each doc, cosine similarity between queyVector and each docVector

    doc_scores = [[np.dot(curDocVec, query_vector), doc] for doc, curDocVec in doc_vectors.items()]
    doc_scores.sort(reverse=True)
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0:
        print("No results found, try again")
        query = input()
        docs = search_tf_idf(query, index)

    return result_docs, doc_scores

In [25]:
def search_tf_idf(query, index):

    query = token_cleaning_text(query)
    docs = set()
    for term in query:
        try:
            # store in term_docs the ids of the docs that contain "term"
            term_docs=[posting[0] for posting in index[term]]

            # docs = docs Union term_docs
            docs |= set(term_docs)
        except:
            #term is not in index
            pass
    docs = list(docs)
    ranked_docs, doc_scores = rank_documents(query, docs, index, idf, tf)
    return ranked_docs, doc_scores

In [26]:
test_queries = [
    "cotton t-shirt men",
    "women winter jacket",
    "sports shoes running",
    "formal shirt slim fit",
    "casual footwear black"
]

for q in test_queries:
    print(f"\nQuery: {q}")
    ranked_docs, _ = search_tf_idf(q, index)
    print(f"Top 5 results for '{q}':\n")
    for i, pid in enumerate(ranked_docs[:5]):
        title = df[df['pid'] == pid]['title'].iloc[0]
        print(f"{i+1}. document_id: {pid}\n   Title: {title}\n")


Query: cotton t-shirt men
Top 5 results for 'cotton t-shirt men':

1. document_id: TSHFHTGBXB3ENHWH
   Title: Solid Men Polo Neck Yellow T-Shirt

2. document_id: TSHFHTGBHE7HGXKT
   Title: Solid Men Polo Neck Yellow T-Shirt

3. document_id: TSHFVR8F4QDGJAP4
   Title: Solid Men Collared Neck Black T-Shirt

4. document_id: TSHFDVE6JNWBGJ2W
   Title: Self Design Men Round Neck White T-Shirt

5. document_id: TSHFGZKNRCNSQ64A
   Title: Printed Men Round Neck White T-Shirt


Query: women winter jacket
Top 5 results for 'women winter jacket':

1. document_id: JCKFWYTHH3B5GKWS
   Title: Full Sleeve Colorblock Women Casual Jacket

2. document_id: JCKFWYNCWEHNZCTJ
   Title: Full Sleeve Striped Women Casual Jacket

3. document_id: JCKFW7ERKBA4BHY3
   Title: Full Sleeve Solid Women Casual Jacket

4. document_id: JCKFVVHNUHUHMYSN
   Title: Full Sleeve Solid Women Casual Jacket

5. document_id: JCKFVVHNHW89CS4E
   Title: Full Sleeve Striped Women Casual Jacket


Query: sports shoes running
Top 5 re

### BM25

We first transform the dataframe into a matrix to have a list of documents containing the list of terms

In [27]:
paragraph_tokens = df["cleaned_title_description_extra_fields"].tolist()

In [28]:
paragraph_tokens[:1]

[['solid',
  'women',
  'multicolor',
  'track',
  'pant',
  'yorker',
  'trackpant',
  'made',
  '100',
  'rich',
  'comb',
  'cotton',
  'give',
  'rich',
  'lookdesign',
  'comfortskin',
  'friendli',
  'fabricitchfre',
  'waistband',
  'great',
  'year',
  'round',
  'use',
  'proudli',
  'made',
  'india',
  'cloth',
  'accessori',
  'bottomwear',
  'cotton',
  'blend',
  'solid',
  'multicolor']]

We instantiate a BM25Okapi model with this list of tokenized documents.

In [29]:
bm25 = BM25Okapi(paragraph_tokens)
print(bm25)

We create a function that filters the dataset to keep only the documents containing all query terms, computes their BM25 scores, and returns the top-n most relevant documents.

In [30]:
def get_top_n(bm25_model, df, query_text, n=10):
    query_tokens = token_cleaning_text(query_text)

    docs_with_all_terms = []
    for i, doc_tokens in enumerate(df["cleaned_title_description_extra_fields"]):
        if all(term in doc_tokens for term in query_tokens):
            docs_with_all_terms.append(i)

    if not docs_with_all_terms:
        print("No documents contain all query terms.")
        return []

    scores = np.array(bm25_model.get_scores(query_tokens))

    filtered_scores = scores[docs_with_all_terms]

    top_n_local = np.argpartition(filtered_scores, -n)[-n:]
    top_n_local = top_n_local[np.argsort(-filtered_scores[top_n_local])]

    top_n_global = [docs_with_all_terms[i] for i in top_n_local]

    return df.iloc[top_n_global]

In [31]:
test_queries = [
    "cotton t-shirt men",
    "women winter jacket",
    "sports shoes running",
    "formal shirt slim fit",
    "casual footwear black"
]

for q in test_queries:
    print(f"\nQuery: {q}")
    #get the indices of the top N documents using the modified get_top_n function
    top_indices = get_top_n(bm25, df, q, n=5)

    if len(top_indices) > 0:
        print(f"Top 5 results for '{q}':\n")
        for i, (original_idx, row_data) in enumerate(top_indices.iterrows()):
            document_pid = row_data['pid']
            document_title = row_data['title']
            print(f"{i+1}. document_id: {document_pid}\n   Title: {document_title}\n")
    else:
        print("No results found for this query.")


Query: cotton t-shirt men
Top 5 results for 'cotton t-shirt men':

1. document_id: TSHFPYXAYEHK4NYW
   Title: Printed Men Round Neck Blue T-Shirt

2. document_id: TSHFZBYYCW9U7ZDN
   Title: Graphic Print Men Round Neck Black T-Shirt

3. document_id: TSHFZBYXRF9CYBFM
   Title: Color Block Men Round Neck Black T-Shirt

4. document_id: TSHFX3MXZPYGKYKH
   Title: Solid Men Round Neck Multicolor T-Shirt

5. document_id: TSHFX28XJBVBKMJE
   Title: Solid Men Round Neck Pink T-Shirt


Query: women winter jacket
Top 5 results for 'women winter jacket':

1. document_id: JCKFVVHNFANGGWNP
   Title: Full Sleeve Solid Women Casual Jacket

2. document_id: JCKFVVHN72FTTT9B
   Title: Full Sleeve Colorblock Women Casual Jacket

3. document_id: JCKFVVHNHW89CS4E
   Title: Full Sleeve Striped Women Casual Jacket

4. document_id: JCKFWYNCWEHNZCTJ
   Title: Full Sleeve Striped Women Casual Jacket

5. document_id: JCKFVVHNUHUHMYSN
   Title: Full Sleeve Solid Women Casual Jacket


Query: sports shoes running


### New score: pepito score


Our score will take into consideration several things, and we will combine them with a linear combination. In the function defined below we can see the weights we have assigned to each property. We give 40% to the cosine similarity that we get apllying tf-idf; 30% to the discount the product has; 30% to the average rating of the product; and finally we will apply a heavy penalitzation if that product is out of stock, reducing the value to 10% of the initial score.

In [32]:
def calculate_pepito_score(cosine_similarity, average_rating, discount, out_of_stock, cosine_factor=0.40, discount_factor=0.30, rating_factor=0.3, stock_penalty=0.9):
    # Normalize average_rating and dicount to have values between 0-1, we need matching scales to have a balanced score
    normalized_average_rating = average_rating / 5.0
    normalized_discount = discount / 100.0

    # Calculate pepito_score with the weights we have decided for each feature
    pepito_score = (
        cosine_factor * cosine_similarity +
        discount_factor * normalized_discount +
        rating_factor * normalized_average_rating
    )

    # we will apply the penalty if the item is not in stock
    if out_of_stock:
        pepito_score *= 1 - stock_penalty

    return pepito_score

We will need a new ranking function that uses the score we have defined. Similar to the ranker used in tf-idf, we will discard documents that do not contain all terms of the query and then compute the pepito score of each of them to obtain a rank.

In [33]:
def rank_documents_pepito(terms, docs, index, idf, tf, original_df, cosine_factor=0.40, discount_factor=0.30, rating_factor=0.3, stock_penalty=0.9):
    result_docs = []
    doc_scores = []

    #for each term in the query, obtain the set of documents that contain it
    docs_with_query_terms = []
    for term in terms:
        if term in index:
            #the term appears in at least one document because it's present in the index
            #get all documents where the term appears
            term_docs = {doc for doc, _ in index[term]}
            docs_with_query_terms.append(term_docs)

    # if no query terms are found in the index, there are no matching documents
    if not docs_with_query_terms:
        print("No query terms in the index")
        return result_docs, doc_scores

    # intersection: keep only documents that contain all query terms
    docs_with_all_query_terms = set.intersection(*docs_with_query_terms)
    if not docs_with_all_query_terms:
        print("No documents with all query terms")
        return result_docs, doc_scores

    #interested only on the element of the docVector corresponding to the query terms
    # remaining elements 0
    doc_vectors = defaultdict(lambda: [0] * len(terms))
    query_vector = [0] * len(terms)

    # compute the norm for the query tf
    query_terms_count = collections.Counter(terms)
    query_norm = la.norm(list(query_terms_count.values()))

    for termIndex, term in enumerate(terms):
        if term not in index:
            continue

        ## Compute tf*idf(normalize TF as done with documents)
        query_vector[termIndex] = query_terms_count[term] / query_norm * idf[term]

        # Generate doc_vectors for matching docs
        for doc_index, (doc, postings) in enumerate(index[term]):
            if doc in docs_with_all_query_terms:
                doc_vectors[doc][termIndex] = tf[term][doc_index] * idf[term]

    # Calculate initial cosine similarity scores
    initial_cosine_scores = []
    for doc, curDocVec in doc_vectors.items():
        dot_product = np.dot(curDocVec, query_vector)
        doc_norm = la.norm(curDocVec)
        query_norm_val = la.norm(query_vector)
        if doc_norm > 0 and query_norm_val > 0:
            cosine_similarity = dot_product / (doc_norm * query_norm_val)
            initial_cosine_scores.append([cosine_similarity, doc])

    # Calculate Pepito Score for each document
    final_ranked_documents = []
    for cos_sim, doc_pid in initial_cosine_scores:
        doc_info = original_df[original_df['pid'] == doc_pid].iloc[0]
        average_rating = doc_info['average_rating']
        discount = doc_info['discount']
        out_of_stock = doc_info['out_of_stock']

        # this takes the default values but can be changed setting the factors to the desired ones
        pepito_score = calculate_pepito_score(cos_sim, average_rating, discount, out_of_stock, cosine_factor, discount_factor, rating_factor, stock_penalty)
        final_ranked_documents.append([pepito_score, doc_pid])

    # Rank documents by Pepito Score
    final_ranked_documents.sort(key=lambda x: x[0], reverse=True)

    result_docs = [x[1] for x in final_ranked_documents]
    doc_scores = final_ranked_documents

    return result_docs, doc_scores

In [34]:
def search_pepito(query, index, original_df, cosine_factor=0.40, discount_factor=0.30, rating_factor=0.3, stock_penalty=0.9):
    query_tokens = token_cleaning_text(query)
    docs = set()

    for term in query_tokens:
        if term in index:
            term_docs = {posting[0] for posting in index[term]}
            docs.update(term_docs)

    docs_list = list(docs)
    ranked_docs, doc_scores = rank_documents_pepito(query_tokens, docs_list, index, idf, tf, original_df, cosine_factor, discount_factor, rating_factor, stock_penalty)
    return ranked_docs, doc_scores

We can test this new score with the same queries we used to test the others. Here we will print more information about each record so we will now if it works correctly and what details does each product have.



In [35]:
test_queries = [
    "cotton t-shirt men",
    "women winter jacket",
    "sports shoes running",
    "formal shirt slim fit",
    "casual footwear black"
]

for q in test_queries:
    ranked_docs_pepito, doc_scores_pepito = search_pepito(q, index, df)
    #here we will print extra information to see if it works
    print(f"\nTop 5 results for query: '{q}' (using Pepito Score)\n")
    for i, (score, pid) in enumerate(doc_scores_pepito[:5]):
        title = df[df['pid'] == pid]['title'].iloc[0]
        average_rating = df[df['pid'] == pid]['average_rating'].iloc[0]
        discount = df[df['pid'] == pid]['discount'].iloc[0]
        out_of_stock = df[df['pid'] == pid]['out_of_stock'].iloc[0]

        print(f"{i+1}. PID: {pid}\n   Title: {title}\n   Pepito Score: {score:.4f}\n   Avg Rating: {average_rating}\n   Discount: {discount}%\n   Out of Stock: {out_of_stock}\n")


Top 5 results for query: 'cotton t-shirt men' (using Pepito Score)

1. PID: BXRFGYUGJCWWZ7UG
   Title: Printed Men Boxer  (Pack of 1)
   Pepito Score: 0.9280
   Avg Rating: 5.0
   Discount: 76%
   Out of Stock: False

2. PID: TSHFVEHX5PURAVHT
   Title: Color Block Men Round Neck Multicolor T-Shirt
   Pepito Score: 0.9130
   Avg Rating: 5.0
   Discount: 71%
   Out of Stock: False

3. PID: TSHFU63CQQPVJWE7
   Title: Printed Men Round Neck Multicolor T-Shirt  (Pack of 3)
   Pepito Score: 0.8920
   Avg Rating: 5.0
   Discount: 64%
   Out of Stock: False

4. PID: TSHFZ2RDSHS8GCHY
   Title: Solid Men Polo Neck Dark Green T-Shirt
   Pepito Score: 0.8800
   Avg Rating: 4.1
   Discount: 78%
   Out of Stock: False

5. PID: TSHFZ2RDMP8TQADG
   Title: Solid Men Polo Neck Light Green T-Shirt
   Pepito Score: 0.8800
   Avg Rating: 4.1
   Discount: 78%
   Out of Stock: False


Top 5 results for query: 'women winter jacket' (using Pepito Score)

1. PID: JCKFMV5X7CFQMNYR
   Title: Sleeveless Solid Wom

## Word2vec and cosine ranking score

To implement the Word2Vec + cosine similarity ranking approach, we followed several steps:

1. Train a Word2Vec model using the cleaned_title_description_extra_fields column of the DataFrame, which contains the tokenized and cleaned text for each document.

2. Generate a single vector representation for each document of the DataFrame by averaging the Word2Vec embeddings of all its cleaned terms. Obtaining a dictionary where keys are PIDs and values are their corresponding vectors.

3. Define the function rank_documents_word2vec_cosine, which computes cosine similarity between the query vector and the document vectors, and ranks only those documents that contain all query terms.

4. Define the function search_word2vec_cosine, which preprocesses the query, generates its Word2Vec vector representation, filters the DataFrame to retain only the documents that contain all query terms in the document dictionary, and then calls the ranking function.

5. Execute the five test queries using this ranking approach and display the top 20 results (PID) returned for each query.

### 1. Train Word2Vec Model


In [36]:
sentences = df['cleaned_title_description_extra_fields'].tolist()

word2vec_model = Word2Vec(sentences, vector_size=100, window=10, min_count=1, negative=15, sg=0)

### 2. Generate a single vector representation for each document

In [37]:
#dictionary to store document vectors, where keys are their PIDs
document_vectors = {}

for idx, row in df.iterrows():
    pid = row['pid']
    terms = row['cleaned_title_description_extra_fields']

    # get all term vectors of the document
    vectors = []
    for term in terms:
        if term in word2vec_model.wv:
            term_vector = word2vec_model.wv[term]
            vectors.append(term_vector)

    if vectors:
        # average the vectors of all words to create a single vector
        document_vectors[pid] = np.mean(vectors, axis=0)
    else:
        # if no terms are in the vocabulary, assign a zero vector
        document_vectors[pid] = np.zeros(word2vec_model.wv.vector_size)


### 3. Define the function rank_documents_word2vec_cosine

In [38]:
def rank_documents_word2vec_cosine(query_vector, document_vectors):
    result_docs = []
    doc_scores = []

    # compute cosine similarity for all the filtered documents
    for pid, doc_vector in document_vectors.items():
        if norm(doc_vector) == 0 or norm(query_vector) == 0:
            score = 0
        else:
            score = np.dot(doc_vector, query_vector) / (norm(doc_vector) * norm(query_vector))

        doc_scores.append([score, pid])

    # sort documents by score descending
    doc_scores.sort(reverse=True)

    # extract ordered PIDs
    result_docs = [x[1] for x in doc_scores]
    if len(result_docs) == 0:
        print("No results found")

    return result_docs, doc_scores


### 4. Define the function search_word2vec_cosine

In [39]:
def search_word2vec_cosine(query, document_vectors):
    # tokenize and clean the query
    query_terms = token_cleaning_text(query)

    # filter documents that contain all query terms (conjunctive)
    valid_pids = []
    for idx, row in df.iterrows():
        doc_terms = row["cleaned_title_description_extra_fields"]
        if all(term in doc_terms for term in query_terms):
            valid_pids.append(row["pid"])

    if not valid_pids:
        print("No documents contain all query terms.")
        return [], []

    # keep only the documents that satisfy the conjunctive query
    filtered_doc_vectors = {pid: document_vectors[pid] for pid in valid_pids}

    # create query vector by averaging Word2Vec embeddings
    query_vecs = []
    for term in query_terms:
        if term in word2vec_model.wv:
            term_vector = word2vec_model.wv[term]
            query_vecs.append(term_vector)

    if query_vecs:
        query_vector = np.mean(query_vecs, axis=0)
    else:
        query_vector = np.zeros(word2vec_model.wv.vector_size)

    # rank filtered documents by cosine similarity
    ranked_docs, doc_scores = rank_documents_word2vec_cosine(query_vector, filtered_doc_vectors)

    return ranked_docs, doc_scores

### 5. Execute the five test queries using this ranking approach

In [40]:
test_queries = [
    "cotton t-shirt men",
    "women winter jacket",
    "sports shoes running",
    "formal shirt slim fit",
    "casual footwear black"
]

print("Proposed test queries:")
for q in test_queries:
    print("-", q)

Proposed test queries:
- cotton t-shirt men
- women winter jacket
- sports shoes running
- formal shirt slim fit
- casual footwear black


In [41]:
for q in test_queries:
    print(f"\nQuery: {q}")
    ranked_docs_word2vec_cosine, _ = search_word2vec_cosine(q, document_vectors)
    print(f"Top results for '{q}': {ranked_docs_word2vec_cosine[:20]}")


Query: cotton t-shirt men
Top results for 'cotton t-shirt men': ['TSHEUUTYSXSH8W8Q', 'TSHEUP779ZGHTYTM', 'TSHF7CPR5WJZYWUY', 'TSHF7CR8RNSG5APJ', 'TSHF7CRTWTJTRAJN', 'TSHF7CQ4PEDTSSKN', 'TSHEVQYFFHSKUURR', 'TSHF743BX74XGNNX', 'TSHFTN8J3X5HCSGJ', 'TSHEUVYHM7CDGDYQ', 'TSHEGGR7AGF3ZFNU', 'TSHFHTGBXB3ENHWH', 'TSHFHTGBHE7HGXKT', 'TSHEUW4QHJ3JYZMR', 'TSHFYJR8YGMFCXKH', 'TSHFHTGKXCZFQV6C', 'TSHFHTGKR6WZPYK2', 'TSHFHTGKJ4R5YZ5H', 'TSHFHTGGKMW9CUBC', 'TSHFHTGGHEHSW2VX']

Query: women winter jacket
Top results for 'women winter jacket': ['JCKFMV5X7CFQMNYR', 'JCKFMV5WTASYK4Y2', 'JCKFGZWZ2K7GS7CT', 'JCKFWZYYYCYPWS6P', 'JCKFWZYYST9KKGVQ', 'JCKFWZYYHRJZ5XCH', 'JCKFYHU4JAY7PSZQ', 'JCKFYPMYKQTAPDGF', 'JCKFFYJ2ZMKAZUNZ', 'JCKFKGR4FADV7SZS', 'JCKFNNHSBFDFB46R', 'JCKFWZYYY3SJ6MGG', 'JCKFWZYYV3VSGNNY', 'JCKFWZYYM7YVUGCF', 'JCKFWZYYFZZMCR38', 'JCKFWZYYDTFZURB4', 'JCKFWZYYZGY3357H', 'JCKFWZYYWSZUEAZU', 'JCKFWZYYJVHCCMEF', 'JCKFWZYY7HHM6F2U']

Query: sports shoes running
Top results for 'sports shoes running